<a href="https://colab.research.google.com/github/JuniorHZ19/HerramientasIA/blob/main/InterpolacionFramesImagen%2BVideo%2BAudioFps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INTERPOLACION CON PRACTICAL RIFE


In [ ]:
#@markdown # **1) Instalar Librerias**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/hzwer/Practical-RIFE.git
%cd Practical-RIFE
! import skvideo.io
!pip3 install -r requirements.txt
!pip3 install --upgrade pip setuptools
!pip install sk-video

In [ ]:
#@markdown # **2) Instalar Modelo**
!mkdir /content/Practical-RIFE/train_log
!gdown --id 1MmaZfFxHVEs0spM4iCB5NDRDM-bWKlIA
!7z e InterpolacionModelo.zip

# Ruta al archivo ZIP que deseas descomprimir
archivo_zip = '/content/Practical-RIFE/InterpolacionModelo.zip'

import zipfile
# Directorio de destino para la descompresión
directorio_destino = '/content/Practical-RIFE/train_log'

with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    # Descomprime los archivos en el directorio de destino
    zip_ref.extractall(directorio_destino)


In [ ]:
#@markdown # **3) Montar Driver**
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
#@markdown # 3.5)** Montar el archivo de Google Drive
#@markdown Pon la ruta del archivo en tu google drive "content/drive/MyDrive/... y da click"
import shutil


drive_mounted = True
drive_path = '/content/drive/MyDrive/10fpstest.mp4'#@param{type: "string"}
file_name = drive_path.split('/')[-1]
input_path = f'/content/drive/MyDrive{drive_path}'
shutil.copy(drive_path,'/content/Practical-RIFE')
print("video path set:\n",input_path)

In [ ]:
#@markdown # **4) ** Interpolar Imagenes
import shutil
import subprocess
import os
fps=60
carpeta_imanges='/content/Practical-RIFE/Imagenes' #@param{type: "string"}
#@markdown  ######Las imagenes tien que seguir un patron como pngs, like input/0.png ... input/612.png)
FormatoSalida='mp4' #@param{type: "string"}
expotencial=4 #@param{type: "integer"}
#@markdown  ###### 2^ 4 =16 si tenemos dos imagnes generar 16 imagnes si tenemos mas imagnes lo elevara a este expotencial tener cuidado al tenre muchos iamgnes

# Ruta de la carpeta que deseas borrar
carpeta_a_borrar = '/content/Practical-RIFE/vid_out'

# Elimina todos los elementos de la carpeta
for elemento in os.listdir(carpeta_a_borrar):
    elemento_completo = os.path.join(carpeta_a_borrar, elemento)
    if os.path.isfile(elemento_completo):
        os.remove(elemento_completo)
    else:
        os.rmdir(elemento_completo)

#Si es resoluacion 4 poner --scale=0.5
!python3 inference_video.py --exp=4  --fps={fps} --img={carpeta_imanges}

cmd = [
      'ffmpeg',

     '-i',
     f'/content/Practical-RIFE/vid_out/%07d.png',

        #'-vf',"scale=3840:2160",
       '-c:v',
       'h264_nvenc',
      '-r',
      '60',
      # '-b:v','5M', #birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)

       f'/content/Practical-RIFE/Imagen.'+FormatoSalida

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Se creo la imageninterpolada."+FormatoSalida)
    shutil.move(f'/content/Practical-RIFE/Imagen.'+FormatoSalida, f'/content/Practical-RIFE/ImagenInterpolada.'+FormatoSalida)



In [ ]:
#@markdown # **3) Inter polacion de video**
#@markdown
import shutil
fps=60
muliplicacion=6

#Si es resoluacion 4 poner --scale=0.5
!python3 inference_video.py --exp=1 --fps={fps}  --multi={muliplicacion} --video={file_name} --scale=1

In [ ]:
#@markdown # **5) ** Poner audio original

import subprocess
import os
name_file_sin_ext=os.path.splitext(file_name)[0]

fileFullNamme= f'{name_file_sin_ext}_{str(muliplicacion)}X_{fps}fps.mp4'
print(fileFullNamme)


if os.path.exists(f'/content/Practical-RIFE/Audio_{fileFullNamme}'):
    os.remove(f'/content/Practical-RIFE/Audio_{fileFullNamme}')

cmd = [
      'ffmpeg',

     '-i',
     f'/content/Practical-RIFE/{fileFullNamme}',
      '-i',
        f'/content/Practical-RIFE/{file_name}',
      '-map', '0:v',
      '-map', '1:a',
        #'-vf',"scale=3840:2160",
       '-c:v',
       'h264_nvenc',
       '-c:a',
        'mp3',
      '-r',
      '60',
       '-b:v','5M', #birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)


       f'/content/Practical-RIFE/Audio_{fileFullNamme}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Se puso el audio ")

In [ ]:
#@markdown # **6)Descargar archivo mejorado**
#@markdown Se pasara el archivo a nuestro drive
import os
from google.colab import files




if drive_mounted:
  if os.path.exists('/content/drive/MyDrive/Real-Rife-Videos') is not True:
    os.makedirs('/content/drive/MyDrive/Real-Rife-Videos')
  shutil.move(f'/content/Practical-RIFE/Audio_{fileFullNamme}',f'/content/drive/MyDrive/Real-Rife-Videos/')
  print(f"MOviendo {fileFullNamme} el archivo a la carpetaReal-Rife-Videos en tu Drive")


# Utils Conversion a 10/20/30 fps (opcional)

In [ ]:
#@markdown #   ** Convertir a 10fps (Opcional Experimental)
#@markdown Si nuestro archvio no esta en 10fps  los videos van a salir mas largos o cortos que no encajaran con el audio

import subprocess

cmd = [
      'ffmpeg',
     '-i',f'/content/Practical-RIFE/{file_name}',

     #'-vf',"scale=3840:2160",

    "-r", '10',  # Establece la tasa de cuadros de salida en 30 fps
    "-c:v", "h264_nvenc",  # Utiliza el códec NVENC H.264
    "-c:a", "aac",  # Códec de audio (puedes ajustarlo según tus necesidades)
    "-strict", "experimental",

      "-b:v","40M",#birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)


       f'/content/Practical-RIFE/10_{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)

    raise RuntimeError(stderr)
else:

  shutil.move(f'/content/Practical-RIFE/10_{file_name}', f'/content/Practical-RIFE/{file_name}')


In [ ]:
#@markdown #  ** Convertir a 20fps (Opcional Experimental)
#@markdown Si nuestro archvio no esta en 20fps  los videos van a salir mas largos o cortos que no encajaran con el audio

import subprocess

cmd = [
      'ffmpeg',
     '-i',f'/content/Practical-RIFE/{file_name}',

     #'-vf',"scale=3840:2160",

    "-r", '20',  # Establece la tasa de cuadros de salida en 30 fps
    "-c:v", "h264_nvenc",  # Utiliza el códec NVENC H.264
    "-c:a", "aac",  # Códec de audio (puedes ajustarlo según tus necesidades)
    "-strict", "experimental",

      #"-b:v","40M",#birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)


       f'/content/Practical-RIFE/20_{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)

    raise RuntimeError(stderr)
else:

  shutil.move(f'/content/Practical-RIFE/10_{file_name}', f'/content/Practical-RIFE/{file_name}')


In [ ]:
#@markdown #  ** Convertir a 30fps (Opcional)
#@markdown Si nuestro archvio no esta en 30fps  los videos van a salir mas largos o cortos que no encajaran con el audio

import subprocess

cmd = [
      'ffmpeg',
     '-i',f'/content/Practical-RIFE/{file_name}',

     #'-vf',"scale=3840:2160",

    "-r", '30',  # Establece la tasa de cuadros de salida en 30 fps
    "-c:v", "h264_nvenc",  # Utiliza el códec NVENC H.264
    "-c:a", "aac",  # Códec de audio (puedes ajustarlo según tus necesidades)
    "-strict", "experimental",

      "-b:v","20M",#birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)


       f'/content/Practical-RIFE/30_{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)

    raise RuntimeError(stderr)
else:

  shutil.move(f'/content/Practical-RIFE/30_{file_name}', f'/content/Practical-RIFE/{file_name}')
